In [4]:
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver import ActionChains
from selenium.webdriver.common.actions.wheel_input import ScrollOrigin
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys

import time
import itertools
import pandas as pd

df = pd.DataFrame(columns=['name', 'year', 'authors', 'journal', 'categories', 'key ideas', 'link'])

url = "https://scholar.google.com/"

keywords = {'optional': ['Energy storage', 'Multistage', 'SDDP', 'rolling horizon'],
            'required': ['disaster']}

min_length = 3
combinations = []

optional = keywords['optional']
for L in range(len(optional) + 1):
    for subset in itertools.combinations(optional, L):
        if len(subset) + len(keywords['required']) >= min_length:
            combinations.append(tuple(keywords['required'] + [s for s in subset]))


min_year = 2022

for c in combinations:#
    op = webdriver.ChromeOptions()
    #op.add_argument('headless')
    driver = webdriver.Chrome(options=op)
    query = ', '.join(c)
    with driver as browser:
        browser.set_window_size(1024, 768)
        browser.get(url)
        action = ActionChains(driver)

        results_div = driver.find_element(By.CSS_SELECTOR, "input[name='q']")
        action.move_to_element(results_div).click().send_keys(query).perform()
        action.move_to_element(results_div).click().send_keys(Keys.ENTER).perform()

        #q_button = driver.find_element(By.CSS_SELECTOR, "#gs_hdr_tsb")

        time.sleep(2)

        soup = BeautifulSoup(browser.page_source, 'html.parser')

        articles = soup.find_all("div", {"class": 'gs_r'})
    
    new_df = pd.DataFrame(columns=['name', 'year', 'authors', 'journal', 'categories', 'key ideas', 'link'], index=[i for i in range(len(articles)-1)])

    for i, article in enumerate(articles):
        try:
            title_data =  article.find('div', {'class': 'gs_ri'}).find('h3').find('a')
            title = title_data.text
            link = title_data['href']

            author_data = article.find('div', {'class': 'gs_ri'}).find('div', {'class': 'gs_a'})
            date = int(author_data.text.split("-")[-2].split(',')[-1].strip())
            journal = author_data.text.split("-")[-1]

            authors_list = author_data.text.split("-")[0][:-1].split(',')

            if len(authors_list) > 3:
                authors = authors_list[0] + " et al."
            else:
                authors = " &".join(authors_list)

            new_row = pd.DataFrame(columns=['name', 'year', 'authors', 'journal', 'categories', 'key ideas', 'link'], index=[0])

            if date >= min_year:
                new_df.iloc[i] = [title, date, authors, journal, query, "", link] 
            
            #new_df = pd.concat(new_df, new_row)
        except:
            continue
    new_df = new_df.dropna()

    df = pd.concat([df, new_df])
    time.sleep(2)

df.to_csv('result.csv')
df

,name,year,authors,journal,categories,key ideas,link
1,Resilient resource allocations for multi-stage...,2024,J Li et al.,ieeexplore.ieee.org,"disaster, Energy storage, Multistage",,https://ieeexplore.ieee.org/abstract/document/...
3,Coordinated robust configuration of soft open ...,2025,W Ma et al.,Elsevier,"disaster, Energy storage, Multistage",,https://www.sciencedirect.com/science/article/...
4,A Multi-Stage Collaborative Optimization Opera...,2024,H Xu et al.,ieeexplore.ieee.org,"disaster, Energy storage, Multistage",,https://ieeexplore.ieee.org/abstract/document/...
5,A novel robust energy storage planning method ...,2025,H Yang et al.,ieeexplore.ieee.org,"disaster, Energy storage, Multistage",,https://ieeexplore.ieee.org/abstract/document/...
6,Multi-stage expansion planning of energy stora...,2022,P Li et al.,Springer,"disaster, Energy storage, Multistage",,https://link.springer.com/article/10.1186/s416...
7,A multi-stage coordinated resilience enhanceme...,2025,W Tang et al.,Elsevier,"disaster, Energy storage, Multistage",,https://www.sciencedirect.com/science/article/...
8,Enhancing resilience and sustainability of dis...,2023,H Saboori,Elsevier,"disaster, Energy storage, Multistage",,https://www.sciencedirect.com/science/article/...
1,Power System Transition Planning: An Industry-...,2025,A A,arxiv.org,"disaster, Energy storage, SDDP",,https://arxiv.org/abs/2505.01331
2,On the Applications of Stochastic Dual Dynamic...,2023,BK SERANILLA,orbilu.uni.lu,"disaster, Energy storage, SDDP",,https://orbilu.uni.lu/handle/10993/58728
5,Multistage operation for distributed solar and...,2023,Y Li et al.,ieeexplore.ieee.org,"disaster, Energy storage, SDDP",,https://ieeexplore.ieee.org/abstract/document/...


In [3]:
df

,name,year,authors,journal,categories,key ideas,link


,level_0,name,year,authors,journal,categories,key ideas,link
0,0,A Multi Stage DRO-SDDP Approach for Planning M...,2025,J Feng et al.,ieeexplore.ieee.org,"disaster, Energy storage, Multistage, SDDP",,https://ieeexplore.ieee.org/abstract/document/...
1,1,Using stochastic dual dynamic programming to s...,2024,A Tabares & P Cortés,mdpi.com,"disaster, Energy storage, Multistage, SDDP",,https://www.mdpi.com/1996-1073/17/11/2628
2,2,Dual stochastic dual dynamic programming for m...,2023,X Zhang et al.,ieeexplore.ieee.org,"disaster, Energy storage, Multistage, SDDP",,https://ieeexplore.ieee.org/abstract/document/...
3,3,Security-Constrained Multi-Stage Robust Dynami...,2025,L Dai et al.,mdpi.com,"disaster, Energy storage, Multistage, SDDP",,https://www.mdpi.com/1996-1073/18/5/1073
4,4,A Stochastic Dual Dynamic Programming Approach...,2024,RA Akinwonmi,search.proquest.com,"disaster, Energy storage, Multistage, SDDP",,https://search.proquest.com/openview/fd8d97d32...
5,5,A multi-stage risk-averse planning approach fo...,2025,J Feng & Z Hu,ieeexplore.ieee.org,"disaster, Energy storage, Multistage, SDDP",,https://ieeexplore.ieee.org/abstract/document/...
6,6,Multi-Stage Stochastic Programming for Disaste...,2025,S Bhattarai,search.proquest.com,"disaster, Energy storage, Multistage, SDDP",,https://search.proquest.com/openview/02b551e16...
7,7,… and storage investments and operation planni...,2023,Y Ermoliev & N Komendantova…,taylorfrancis.com,"disaster, Energy storage, Multistage, rolling ...",,https://www.taylorfrancis.com/chapters/edit/10...
8,8,A Two-Stage Robust Approach for Resilient Unit...,2024,X Yang et al.,ieeexplore.ieee.org,"disaster, Energy storage, Multistage, rolling ...",,https://ieeexplore.ieee.org/abstract/document/...
9,9,Multi-time-step rolling optimization strategy ...,2022,H Wan et al.,ieeexplore.ieee.org,"disaster, Energy storage, Multistage, rolling ...",,https://ieeexplore.ieee.org/abstract/document/...


In [117]:
**(keywords['required'] + [s for s in subset])

SyntaxError: invalid syntax (1856430020.py, line 1)